# Sistemas de recomendação

## O que são?

* sistemas que calculam e fornecem itens relevantes a um usuário baseado no conhecimento sobre o usuário, conteúdo e sobre as interações do usuário com os itens (tradução livre de uma definição encontrada em [[1]](https://www.manning.com/books/practical-recommender-systems))


## Importância hoje?

* anedotas
    1. youtube
    2. arxiv

## Exemplos:

* recomendações da Amazon após visualização do livro _Foundations of Natural Language Processing_:

![](resources/imgs/amazon_recommender.png)

* recomendações da Netflix para mim:

![](resources/imgs/netflix_recommender.png)

* **Mais??**

## Quando falamos de sistemas de recomendação...

... devemos definir:

* o quê?

* por quê?

* onde/como?

* quão personalizado?

## Efeitos indesejados

* invasão de privacidade

<img src="resources/imgs/target.png" width="500">

Fonte: [Business Insider](https://www.businessinsider.com/the-incredible-story-of-how-target-exposed-a-teen-girls-pregnancy-2012-2)

* espalhamento de desinformação

<img src="resources/imgs/youtube.png" width="300">

Fonte: [Twitter](https://twitter.com/PeterD_Adams/status/1089583479761788928). Leia mais no [NYT](https://www.nytimes.com/2018/03/10/opinion/sunday/youtube-politics-radical.html#click=https://t.co/cGE67FhlsI).

* quebra de confiança

<img src="resources/imgs/taboo.png" width="600">

Fonte: essas são as recomendações geradas pelo Taboola no final da página do [Business Insider](https://www.businessinsider.com/the-incredible-story-of-how-target-exposed-a-teen-girls-pregnancy-2012-2). Leia mais sobre esse tipo de recomendação em [Fortune](http://fortune.com/2016/11/01/outbrain-taboola/).

## Criando um sistema de recomendação

Nas atividades dessa aula, vamos usar o dataset [goodbooks-10k](https://github.com/zygmuntz/goodbooks-10k), um dataset que contém as avaliações (_ratings_) para os 10 mil livros mais populares no site [goodreads](https://www.goodreads.com/).

Nosso objetivo será criar um sistema de recomendação assertivo; mais especificamente, queremos maximizar a leitura dos livros que recomendamos aos usuários.

### Carregando o dataset

Se você não fez o download do dataset ainda, rode a célula abaixo após descomentá-la (remova o `#`).

In [18]:
#!git clone https://github.com/zygmuntz/goodbooks-10k.git resources/data

Cloning into 'resources/data'...
remote: Enumerating objects: 66, done.
remote: Total 66 (delta 0), reused 0 (delta 0), pack-reused 66
Unpacking objects: 100% (66/66), done.


In [1]:
import pandas as pd

In [2]:
user_ratings_df = pd.read_csv('resources/data/ratings.csv')

In [3]:
user_ratings_df.head(n=1)

,user_id,book_id,rating
0,1,258,5


In [4]:
print('Informações sobre o dataset:\n')
print('* Quantidade de avaliações: {:,}'.format(len(user_ratings_df)))
print('* Quantidade de livros com avaliação: {:,}'.format(len(user_ratings_df['book_id'].unique())))
print('* Quantidade de usuários únicos: {: ,}'.format(len(user_ratings_df['user_id'].unique())))

Informações sobre o dataset:

* Quantidade de avaliações: 5,976,479
* Quantidade de livros com avaliação: 10,000
* Quantidade de usuários únicos:  53,424


#### Vamos separar parte do nosso dataset como treino e parte dele como teste

In [5]:
from sklearn.model_selection import train_test_split

In [22]:
train_df, test_df = train_test_split(user_ratings_df)

* vamos checar se a quantidade de usuários no dataset de treino é a mesma do dataset original

In [40]:
len(train_df['user_id'].unique()) == len(user_ratings_df['user_id'].unique())

True

### Filtro colaborativo

Há dois tipos de filtro colaborativo: o baseado em usuário (`user user`) e o baseado no conteúdo (`item item`).

<img src="resources/imgs/collaborative_filtering.png" width="400">

#### **Atividade**: construção de um filtro colaborativo baseado em usuário

Etapas:

1. construir uma matriz de usuários (`user_id`) por itens (`book_id`)
2. normalizar essa matriz: para cada `rating`, será subtraída a média de `ratings` do usuário
3. encontrar os grupos de usuários similares usando o algoritmo [knn](https://scikit-learn.org/stable/modules/neighbors.html)
4. construir uma função que, dado o id de um usuário, traz $n$ recomendações para ele

** Tempo estimado para a atividade: ** 15 minutos

**Etapa 1: ** vamos construir uma matriz de usuários por itens

In [15]:
user_items_matrix = pd.pivot_table(
    user_ratings_df,
    values='rating',
    index=['user_id'],
    columns=['book_id'],
    aggfunc=lambda x: x
).fillna(0).reset_index().values

MemoryError: 

* para evitar erros de memória, você pode construir uma [matriz esparsa](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) com os dados!

In [27]:
import numpy as np
from scipy.sparse import coo_matrix

In [28]:
unique_user_ids_df = train_df[['user_id']].drop_duplicates()
unique_user_ids_df['id'] = np.arange(len(unique_user_ids_df))

In [29]:
train_df = train_df.merge(unique_user_ids_df, on='user_id', how='inner')

In [34]:
min(train_df['book_id'].unique()), max(train_df['book_id'].unique()), len(train_df['book_id'].unique())

(1, 10000, 10000)

In [35]:
data = train_df['rating'].values
row = train_df['id'].values
col = (train_df['book_id'] - 1).values
user_items_matrix = coo_matrix(data, (row, col))

In [39]:
user_items_matrix

<1x4482359 sparse matrix of type '<class 'numpy.int64'>'
	with 4482359 stored elements in COOrdinate format>

## Fatorização de matrizes

Também podemos construir um sistema de recomendação usando técnicas de fatorização de matrizes. A ideia de fatorizar matrizes é reduzir a dimensionalidade e, consequentemente, a esparsidade das matrizes de interações, para gerar melhores recomendações para os usuários.

### SVD (Singular Value Decomposition)

### NMF (Non-negative Matrix Factorization)

## Bônus: LDA

Acabamos de reproduzir parte da evolução do [sistema de recomendação do NYT](https://open.blogs.nytimes.com/2015/08/11/building-the-next-new-york-times-recommendation-engine/).

## Mais Bônus: ranking e avaliação